<a href="https://colab.research.google.com/github/yamanoyu/atcorder/blob/master/Educational_DP_Contest_DP_%E3%81%BE%E3%81%A8%E3%82%81%E3%82%B3%E3%83%B3%E3%83%86%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A - Frog 1

In [7]:
n = int(input())
h_list = list(map(int, input().split()))

INF = float('inf')

dp = [INF] * n
dp[0] = 0

for i in range(n):
  if i == 0:
    continue
  elif i == 1:
    dp[i] = abs(h_list[i] - h_list[i - 1])
  else:
    dp[i] = min(dp[i - 1] + abs(h_list[i] - h_list[i - 1]), dp[i - 2] + abs(h_list[i] - h_list[i - 2]))

ans = dp[-1]
print(ans)

6
30 10 60 10 60 50
40


## B - Frog 2

In [23]:
n, k = map(int, input().split())
h_list = list(map(int, input().split()))

INF = float('inf')

dp = [INF] * n
dp[0] = 0

for i in range(n):
  if i == 0:
    continue
  else:
    for j in range(0, min(i + 1, k + 1)):
      dp[i] = min(dp[i], dp[i - j] + abs(h_list[i] - h_list[i - j]))

ans = dp[-1]
print(ans)

10 4
40 10 20 70 80 10 20 70 80 60
40
